In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

# model1

In [2]:
train = pd.read_csv('train_zero_new.csv')

train.shape
train.head()

(1597063, 344)

,instance_id,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_hot,user_hot,shop_hot,brand_hot,occupation_hot,city_hot,item_trade_hot,user_trade_hot,shop_trade_hot,brand_trade_hot,occupation_trade_hot,city_trade_hot,user_sell_power,user_sell_power_mean,user_sell_power_mean_item_minus,user_sell_trade_power,user_sell_trade_power_mean,user_sell_trade_power_mean_item_minus,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,user_minute_query,item_day_query,item_hour_query,item_minute_query,user_item_day_query,user_item_hour_query,user_item_minute_query,min_time,user_is_his,item_is_his,brand_is_his,user_is_trade_his,item_is_trade_his,brand_is_trade_his,item_trade_percent,brand_trade_percent,user_trade_percent,occupation_trade_percent,is_home,shop_item_count,brand_item_count,city_brand_count,city_item_count,shop_item_count_his,brand_item_count_his,city_brand_count_his,city_item_count_his,time_6,day_6,hour_6,minute_6,item_category_split_count_6,item_property_split_count_6,pred_category_split_count_6,pred_property_split_count_6,item_pred_category_score_6,item_pred_property_score_6,item_pred_category_score_item%_6,item_pred_property_score_item%_6,item_pred_category_score_pred%_6,item_pred_property_score_pred%_6,user_shop_count_6,user_item_count_6,user_context_count_6,user_shop_trade_count_6,user_item_trade_count_6,user_context_trade_count_6,user_brand_count_6,user_city_count_6,user_brand_trade_count_6,user_city_trade_count_6,item_occupation_count_6,item_age_count_6,item_gender_count_6,item_occupation_trade_count_6,item_age_trade_count_6,item_gender_trade_count_6,shop_score_6,item_score_6,item_score2_6,item_score3_6,item_hot_6,user_hot_6,shop_hot_6,brand_hot_6,occupation_hot_6,city_hot_6,item_trade_hot_6,user_trade_hot_6,shop_trade_hot_6,brand_trade_hot_6,occupation_trade_hot_6,city_trade_hot_6,user_sell_power_6,user_sell_power_mean_6,user_sell_power_mean_item_minus_6,user_sell_trade_power_6,user_sell_trade_power_mean_6,user_sell_trade_power_mean_item_minus_6,yesterday_6,yesterhour_6,user_day_query_6,user_yesterday_query_6,user_hour_query_6,user_yesterhour_query_6,user_minute_query_6,item_day_query_6,item_hour_query_6,item_minute_query_6,user_item_day_query_6,user_item_hour_query_6,user_item_minute_query_6,min_time_6,user_is_his_6,item_is_his_6,brand_is_his_6,user_is_trade_his_6,item_is_trade_his_6,brand_is_trade_his_6,item_trade_percent_6,brand_trade_percent_6,user_trade_percent_6,occupation_trade_percent_6,is_home_6,shop_item_count_6,brand_item_count_6,city_brand_count_6,city_item_count_6,shop_item_count_his_6,brand_item_count_his_6,city_brand_count_his_6,city_item_count_his_6,item_category_list_full_time,item_property_list_full_time,predict_category_property_full_time,time_full_time,day_full_time,hour_full_time,minute_full_time,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,minute,user_query_day,user_query_day_hour,user_query_minute,user_count,user_day_count,user_hour_count,user_minute_count,user_next_count,us

In [3]:
train = train.select_dtypes(include=['int64', 'float64'])

In [5]:
origin = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level',
    'user_occupation_id', 'user_star_level', 
    'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

features_6 = [
    'user_shop_count_6', 'user_item_count_6', 'user_context_count_6',
    'user_shop_trade_count_6', 'user_item_trade_count_6',
    'user_context_trade_count_6', 'user_brand_count_6', 'user_city_count_6',
    'user_brand_trade_count_6', 'user_city_trade_count_6',
    'item_occupation_count_6', 'item_age_count_6', 'item_gender_count_6',
    'item_occupation_trade_count_6', 'item_age_trade_count_6',
    'item_gender_trade_count_6', 'shop_score_6', 'item_score_6',
    'item_score2_6', 'item_score3_6', 'item_hot_6', 'user_hot_6', 'shop_hot_6',
    'brand_hot_6', 'occupation_hot_6', 'city_hot_6', 'item_trade_hot_6',
    'user_trade_hot_6', 'shop_trade_hot_6', 'brand_trade_hot_6',
    'occupation_trade_hot_6', 'city_trade_hot_6', 'user_sell_power_6',
    'user_sell_power_mean_6', 'user_sell_power_mean_item_minus_6',
    'user_sell_trade_power_6', 'user_sell_trade_power_mean_6',
    'user_sell_trade_power_mean_item_minus_6', 'yesterday_6', 'yesterhour_6',
    'user_day_query_6', 'user_yesterday_query_6', 'user_hour_query_6',
    'user_yesterhour_query_6', 'user_minute_query_6', 'item_day_query_6',
    'item_hour_query_6', 'item_minute_query_6', 'user_item_day_query_6',
    'user_item_hour_query_6', 'user_item_minute_query_6', 'min_time_6',
    'user_is_his_6', 'item_is_his_6', 'brand_is_his_6', 'user_is_trade_his_6',
    'item_is_trade_his_6', 'brand_is_trade_his_6', 'user_count_6',
    'user_day_count_6', 'user_hour_count_6', 'user_minute_count_6',
    'item_trade_percent_6', 'brand_trade_percent_6', 'user_trade_percent_6',
    'occupation_trade_percent_6', 'is_home_6', 'shop_item_count_6',
    'brand_item_count_6', 'city_brand_count_6', 'city_item_count_6',
    'shop_item_count_his_6', 'brand_item_count_his_6',
    'city_brand_count_his_6', 'city_item_count_his_6', 'next_time_sub_6',
    'last_time_sub_6', 'user_item_num_6', 'user_shop_num_6',
    'user_item_day_num_6', 'user_shop_day_num_6', 'user_item_next_time_sub_6',
    'user_item_last_time_sub_6', 'user_shop_next_time_sub_6',
    'user_shop_last_time_sub_6', 'user_count_first_time_sub_6',
    'user_item_count_first_time_sub_6', 'user_shop_count_first_time_sub_6',
    'max_click_6', 'is_last_click_6', 'max_user_item_click_6',
    'is_last_user_item_click_6', 'max_click_time_sub_6',
    'max_user_item_click_time_sub_6'
]

by = ['instance_id']

target = ['is_trade']

In [6]:
sub_features = []
for i in features_6:
    tmp = str(i[:-2] + '_pre5_6_sub')
    train[tmp] = abs(train[i[:-2]] - train[i])
    sub_features.append(tmp)
len(sub_features)

94

In [7]:
predictors = list(set(train.columns) - set(origin) - set(by + origin))
len(predictors)

380

In [8]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb
# import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/home/user/cuser/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
test = train[train.is_trade == -1]
train = train[train.is_trade != -1]

test.shape
train.shape

(519888, 396)

(1077175, 396)

In [10]:
#baseline
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

tmp_predictors = origin.copy()
len(tmp_predictors)
xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=300,
    nfold=3,
    early_stopping_rounds=50,
    seed=201803)
    
print('Baseline->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))

15

Baseline->300:0.17820566666666668


In [18]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

# feature_peoformance = {}

for i in enumerate(sub_features):
    if i[1] not in feature_peoformance.keys():
        tmp_predictors = origin.copy()
        tmp_predictors.append(i[1])
        xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

        cv = xgb.cv(
            params,
            dtrain=xgbtrain,
            num_boost_round=300,
            nfold=3,
            early_stopping_rounds=50,
            seed=201803)
    
        feature_peoformance[i[1]] = cv.iloc[-1, :]['test-logloss-mean']
        print(str(i[0]) + ':' + str(i[1]) + '->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))

0:user_shop_count_pre5_6_sub->300:0.17810133333333333
3:item_occupation_count_pre5_6_sub->300:0.178168
5:item_occupation_trade_count_pre5_6_sub->300:0.17798633333333333
8:item_hot_pre5_6_sub->300:0.17812933333333333
12:city_hot_pre5_6_sub->300:0.17810099999999998
13:item_trade_hot_pre5_6_sub->300:0.17782966666666666
20:user_sell_power_mean_item_minus_pre5_6_sub->300:0.17806966666666665
23:brand_trade_percent_pre5_6_sub->300:0.17795833333333333
27:city_item_count_his_pre5_6_sub->300:0.17807366666666669
30:user_shop_next_time_sub_pre5_6_sub->300:0.17797933333333335


In [22]:
feature_peoformance

{'brand_hot_6': 0.17812033333333332,
 'brand_hot_pre5_6_sub': 0.17813266666666663,
 'brand_is_his': 0.17820066666666667,
 'brand_is_his_6': 0.17820066666666667,
 'brand_is_trade_his_6': 0.17818433333333336,
 'brand_is_trade_his_pre5_6_sub': 0.17818666666666663,
 'brand_item_count': 0.17815433333333333,
 'brand_item_count_6': 0.17815433333333333,
 'brand_item_count_his_6': 0.178137,
 'brand_item_count_his_pre5_6_sub': 0.178131,
 'brand_trade_hot_pre5_6_sub': 0.1781,
 'brand_trade_percent_6': 0.17802199999999999,
 'brand_trade_percent_pre5_6_sub': 0.17795833333333333,
 'city_brand_count': 0.17808900000000003,
 'city_brand_count_his_6': 0.17809366666666668,
 'city_brand_count_his_pre5_6_sub': 0.178065,
 'city_hot': 0.178111,
 'city_hot_6': 0.17810933333333334,
 'city_hot_pre5_6_sub': 0.17810099999999998,
 'city_item_count': 0.17808633333333335,
 'city_item_count_6': 0.17808633333333335,
 'city_item_count_his': 0.17809666666666665,
 'city_item_count_his_6': 0.17809633333333333,
 'city_item

In [23]:
good_features = []
baseline = 0.17820566666666668

for i in sorted(feature_peoformance.items(), key=lambda x:x[1], reverse=False):
    if i[1] < baseline:
        good_features.append(i[0])
len(good_features)

278

In [24]:
good_features

['is_trade',
 'max_click_time_sub_full_time',
 'max_click_time_sub_6',
 'next_time_sub_6',
 'is_last_click_6',
 'is_last_click',
 'is_last_click_full_time',
 'next_time_sub_full_time',
 'max_user_item_click_time_sub_full_time',
 'max_user_item_click_time_sub',
 'is_last_user_item_click',
 'is_last_user_item_click_full_time',
 'user_query_minute_full_time',
 'user_query_minute_6',
 'user_query_minute',
 'user_minute_query',
 'user_item_next_time_sub_6',
 'user_item_next_time_sub_full_time',
 'user_item_next_time_sub',
 'max_click',
 'user_query_day',
 'max_click_6',
 'user_day_query',
 'user_query_day_full_time',
 'user_query_day_hour',
 'user_hour_query',
 'user_query_day_hour_6',
 'user_shop_next_time_sub_6',
 'next_time_sub_pre5_6_sub',
 'user_shop_next_time_sub',
 'item_trade_percent',
 'user_item_count_first_time_sub_full_time',
 'user_item_count_first_time_sub',
 'user_shop_count_first_time_sub_6',
 'user_item_last_time_sub_6',
 'user_item_last_time_sub_full_time',
 'user_item_num

In [19]:
len(feature_peoformance)

295

In [16]:
sub_features = [
    'user_shop_count_pre5_6_sub',
    'user_item_count_pre5_6_sub', 'user_brand_count_pre5_6_sub',
    'item_occupation_count_pre5_6_sub', 'item_gender_count_pre5_6_sub',
    'item_occupation_trade_count_pre5_6_sub',
    'item_age_trade_count_pre5_6_sub', 'item_gender_trade_count_pre5_6_sub',
    'item_hot_pre5_6_sub', 'user_hot_pre5_6_sub', 'shop_hot_pre5_6_sub',
    'brand_hot_pre5_6_sub', 'city_hot_pre5_6_sub', 'item_trade_hot_pre5_6_sub',
    'user_trade_hot_pre5_6_sub', 'shop_trade_hot_pre5_6_sub',
    'brand_trade_hot_pre5_6_sub', 'city_trade_hot_pre5_6_sub',
    'user_sell_power_pre5_6_sub', 'user_sell_power_mean_pre5_6_sub',
    'user_sell_power_mean_item_minus_pre5_6_sub',
    'brand_is_trade_his_pre5_6_sub', 'item_trade_percent_pre5_6_sub',
    'brand_trade_percent_pre5_6_sub', 'shop_item_count_his_pre5_6_sub',
    'brand_item_count_his_pre5_6_sub', 'city_brand_count_his_pre5_6_sub',
    'city_item_count_his_pre5_6_sub', 'next_time_sub_pre5_6_sub',
    'user_item_next_time_sub_pre5_6_sub', 'user_shop_next_time_sub_pre5_6_sub'
]
len(sub_features)

31

# model2

In [2]:
train = pd.read_csv('train_zero_model2_new.csv')

train.shape
train.head()

(2877748, 208)

,instance_id,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_hot,user_hot,shop_hot,brand_hot,occupation_hot,city_hot,item_trade_hot,user_trade_hot,shop_trade_hot,brand_trade_hot,occupation_trade_hot,city_trade_hot,user_sell_power,user_sell_power_mean,user_sell_power_mean_item_minus,user_sell_trade_power,user_sell_trade_power_mean,user_sell_trade_power_mean_item_minus,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,user_minute_query,item_day_query,item_hour_query,item_minute_query,user_item_day_query,user_item_hour_query,user_item_minute_query,min_time,user_is_his,item_is_his,brand_is_his,user_is_trade_his,item_is_trade_his,brand_is_trade_his,item_trade_percent,brand_trade_percent,user_trade_percent,occupation_trade_percent,is_home,shop_item_count,brand_item_count,city_brand_count,city_item_count,shop_item_count_his,brand_item_count_his,city_brand_count_his,city_item_count_his,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,minute,user_query_day,user_query_day_hour,user_query_minute,user_count,user_day_count,user_hour_count,user_minute_count,user_next_count,user_last_count,user_next_time,user_last_time,next_time_sub,last_time_sub,user_item_num,user_shop_num,user_item_day_num,user_shop_day_num,user_item_num_next,user_item_num_last,user_shop_num_next,user_shop_num_last,user_item_next_time,user_item_last_time,user_shop_next_time,user_shop_last_time,user_item_next_time_sub,user_item_last_time_sub,user_shop_next_time_sub,user_shop_last_time_sub,user_fisrt_num,user_item_fisrt_num,user_shop_fisrt_num,user_count_first_time,user_item_count_first_time,user_shop_count_first_time,user_count_first_time_sub,user_item_count_first_time_sub,user_shop_count_first_time_sub,max_click,is_last_click,max_user_item_click,is_last_user_item_click,max_click_time,max_user_item_click_time,max_click_time_sub,max_user_item_click_time_sub,instance_id_full_time,item_category_list_full_time,item_property_list_full_time,predict_category_property_full_time,user_query_day_full_time,user_query_day_hour_full_time,user_query_minute_full_time,user_count_full_time,user_day_count_full_time,user_hour_count_full_time,user_minute_count_full_time,user_next_count_full_time,user_last_count_full_time,user_next_time_full_time,user_last_time_full_time,next_time_sub_full_time,last_time_sub_full_time,user_item_num_full_time,user_shop_num_full_time,user_item_day_num_full_time,user_shop_day_num_full_time,user_item_num_next_full_time,user_item_num_last_full_time,user_shop_num_next_full_time,user_shop_num_last_full_time,user_item_next_time_full_time,user_item_last_time_full_time,user_shop_next_time_full_time,user_shop_last_time_full_time,user_item_next_time_sub_full_time,user_item_last_time_sub_full_time,user_shop_next_time_sub_full_time,user_shop_last_time_sub_full_time,user_fisrt_num_full_time,user_item_fisrt_num_full_time,user_shop_fisrt_num_full_time,user_count_first_time_full_time,user_item_count_first_time_full_time,user_shop_count_first_time_full_ti

In [3]:
origin = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_star_level',
    'context_page_id', 'shop_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

features = [
    'day', 'hour', 'minute', 'user_query_day', 'user_query_day_hour',
    'user_query_minute', 'item_category_split_count',
    'item_property_split_count', 'pred_category_split_count',
    'pred_property_split_count', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%', 'user_shop_count', 'user_item_count',
    'user_context_count', 'user_shop_trade_count', 'user_item_trade_count',
    'user_context_trade_count', 'user_brand_count', 'user_city_count',
    'user_brand_trade_count', 'user_city_trade_count', 'item_occupation_count',
    'item_age_count', 'item_gender_count', 'item_occupation_trade_count',
    'item_age_trade_count', 'item_gender_trade_count', 'shop_score',
    'item_score', 'item_score2', 'item_score3', 'item_hot', 'user_hot',
    'shop_hot', 'brand_hot', 'occupation_hot', 'city_hot', 'item_trade_hot',
    'user_trade_hot', 'shop_trade_hot', 'brand_trade_hot',
    'occupation_trade_hot', 'city_trade_hot', 'user_sell_power',
    'user_sell_power_mean', 'user_sell_power_mean_item_minus',
    'user_sell_trade_power', 'user_sell_trade_power_mean',
    'user_sell_trade_power_mean_item_minus', 'yesterday', 'yesterhour',
    'user_day_query', 'user_yesterday_query', 'user_hour_query',
    'user_yesterhour_query', 'user_minute_query', 'item_day_query',
    'item_hour_query', 'item_minute_query', 'user_item_day_query',
    'user_item_hour_query', 'user_item_minute_query', 'min_time',
    'user_is_his', 'item_is_his', 'brand_is_his', 'user_is_trade_his',
    'item_is_trade_his', 'brand_is_trade_his', 'user_count', 'user_day_count',
    'user_hour_count', 'user_minute_count', 'item_trade_percent',
    'brand_trade_percent', 'user_trade_percent', 'occupation_trade_percent',
    'is_home', 'shop_item_count', 'brand_item_count', 'city_brand_count',
    'city_item_count', 'shop_item_count_his', 'brand_item_count_his',
    'city_brand_count_his', 'city_item_count_his', 'next_time_sub',
    'last_time_sub', 'user_item_num', 'user_shop_num', 'user_item_day_num',
    'user_shop_day_num', 'user_item_next_time_sub', 'user_item_last_time_sub',
    'user_shop_next_time_sub', 'user_shop_last_time_sub',
    'user_count_first_time_sub', 'user_item_count_first_time_sub',
    'user_shop_count_first_time_sub', 'max_click', 'is_last_click',
    'max_user_item_click', 'is_last_user_item_click', 'max_click_time_sub',
    'max_user_item_click_time_sub'
]

features_fulltime = [
    'next_time_sub_full_time', 'last_time_sub_full_time',
    'user_shop_num_full_time',
    'user_item_day_num_full_time', 'user_shop_day_num_full_time',
    'user_item_next_time_sub_full_time', 'user_item_last_time_sub_full_time',
    'user_shop_next_time_sub_full_time', 'user_shop_last_time_sub_full_time',
    'user_count_first_time_sub_full_time',
    'user_item_count_first_time_sub_full_time',
    'user_shop_count_first_time_sub_full_time', 'max_click_full_time',
    'is_last_click_full_time', 'max_user_item_click_full_time',
    'is_last_user_item_click_full_time', 'max_user_item_click_time_sub_full_time'
]

target = ['is_trade']

In [11]:
train = train[train.is_trade != -1]

train.shape

(2357860, 208)

In [6]:
predictors = features + features_fulltime

len(predictors)
len(set(predictors) & set(train.columns))

127

127

In [7]:
import lightgbm as lgb

In [14]:
params = {
    'objective':'binary',
        # metric='binary_error',
        'num_leaves':7,
        'max_depth':3,
        'learning_rate':0.05,
        'reg_alpha' : .4,
        'reg_lambda' : .2,
        'random_state':1024,
        'colsample_bytree':0.8,
        'subsample':0.9,
#         'n_estimators':20000,
        'n_jobs': 31,
#         'device':'gpu',
        'histogram_pool_size':30270,
        'max_bin':63,
        'gpu_use_dp':True
}

tmp_predictors = origin.copy()
        
lgb_train = lgb.Dataset(train[tmp_predictors],label = train['is_trade'])
cv = lgb.cv(params,lgb_train,num_boost_round=400,nfold=3,early_stopping_rounds=50,seed=1024,verbose_eval=0)
    
feature_peoformance[i[1]] = cv['binary_logloss-mean'][-1]
print('Baseline->' + str(len(cv['binary_logloss-mean'])) + ':' + str(cv['binary_logloss-mean'][-1]))

Baseline->400:0.05386270784670991


In [17]:
params = {
    'objective':'binary',
        # metric='binary_error',
        'num_leaves':7,
        'max_depth':3,
        'learning_rate':0.05,
        'reg_alpha' : .4,
        'reg_lambda' : .2,
        'random_state':1024,
        'colsample_bytree':0.8,
        'subsample':0.9,
#         'n_estimators':20000,
        'n_jobs': 31,
#         'device':'gpu',
        'histogram_pool_size':30270,
        'max_bin':63,
        'gpu_use_dp':True
}

# feature_peoformance = {}

for i in enumerate(predictors):
    if i[1] not in feature_peoformance.keys():
        tmp_predictors = origin.copy()
        tmp_predictors.append(i[1])
        
        lgb_train = lgb.Dataset(train[tmp_predictors],label = train['is_trade'])
        cv = lgb.cv(params,lgb_train,num_boost_round=400,nfold=3,early_stopping_rounds=50,seed=1024,verbose_eval=0)
    
        feature_peoformance[i[1]] = cv['binary_logloss-mean'][-1]
        print(str(i[0]) + ':' + str(i[1]) + '->' + str(len(cv['binary_logloss-mean'])) + ':' + str(cv['binary_logloss-mean'][-1]))

86:city_item_count->400:0.053835117310156
87:shop_item_count_his->400:0.05373102301631652
88:brand_item_count_his->400:0.053810659736875134
89:city_brand_count_his->400:0.05383291575895832
90:city_item_count_his->400:0.05383658630299695
91:next_time_sub->400:0.05305935923041936
92:last_time_sub->400:0.05379531359492929


KeyboardInterrupt: 

In [18]:
feature_peoformance

{'brand_hot': 0.05380854386836101,
 'brand_is_his': 0.05386270784670993,
 'brand_is_trade_his': 0.053864576064782545,
 'brand_item_count': 0.05380849548292038,
 'brand_item_count_his': 0.053810659736875134,
 'brand_trade_hot': 0.053752570562526546,
 'brand_trade_percent': 0.05331861342269145,
 'city_brand_count': 0.05386270784670991,
 'city_brand_count_his': 0.05383291575895832,
 'city_hot': 0.05383569517940337,
 'city_item_count': 0.053835117310156,
 'city_item_count_his': 0.05383658630299695,
 'city_trade_hot': 0.05383294254153451,
 'day': 0.05386216771778785,
 'hour': 0.053688611933397856,
 'is_home': 0.05386882229359266,
 'item_age_count': 0.053863905388363935,
 'item_age_trade_count': 0.053609386104554346,
 'item_category_split_count': 0.0538665804728835,
 'item_day_query': 0.05384627499439543,
 'item_gender_count': 0.05387224846141967,
 'item_gender_trade_count': 0.053383637851312095,
 'item_hot': 0.053858822259746086,
 'item_hour_query': 0.05383057836280197,
 'item_is_his': 0.05

In [20]:
rubbish_features = []
baseline = 0.05386270784670991

for i in sorted(feature_peoformance.items(), key=lambda x:x[1], reverse=False):
    if i[1] > baseline:
        rubbish_features.append(i[0])
len(rubbish_features)

31

In [24]:
len(list(set(predictors) - set(rubbish_features)))

96